In [ ]:
# Installation of packages

!pip install transformers

!pip install datasets

!pip install rouge


In [17]:
# Importing Packages
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer, TrainingArguments, Trainer
from datasets import list_datasets, load_dataset
from rouge import Rouge
from google.colab import drive
import numpy as np
from transformers import BartForConditionalGeneration, BartTokenizer

from google.colab import drive
from datasets import load_dataset, DatasetDict
from transformers import BartForConditionalGeneration, BartTokenizer
import datasets

import numpy as np
from datasets import load_dataset
from transformers import BartTokenizer

import torch
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer, TrainingArguments, Trainer
from transformers import BartConfig, BartForConditionalGeneration
from torch.nn import ReLU, GELU
from torch.nn import functional as F
import torch.nn as nn
import datasets
import torch
from torch.nn import ELU
import torch.nn.functional as F
from transformers import AdamW


In [ ]:
# Load the dataset

dataset = load_dataset('cnn_dailymail', "3.0.0")
rouge = Rouge(metrics=['rouge-1','rouge-2','rouge-l'])

print(dataset)

In [ ]:
# Load the model and the tokenizer

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
# Calculating the rouge score for Pre - Trained Model

generated_summaries,reference_summaries = [],[]
# Loop over the dataset and generate summaries
for example in dataset["train"]:
    # Tokenize the input text and truncate it to a maximum length of 1024 tokens
    input_ids = tokenizer.encode(example["article"], max_length=1024, truncation=True, return_tensors="pt")
    # Generate the summary
    input_ids = input_ids.to(device)
    summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, max_length=142, min_length=56, no_repeat_ngram_size=3)
    summary_ids = summary_ids.to('cpu')
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    generated_summaries.append(summary)
    reference_summaries.append(example["highlights"])

# Calculate ROUGE scores for each summary pair
scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores and their averages for the dataset
for metric, values in scores.items():
    print(f"ROUGE-{metric} scores:\n")
    print(values)
    print(f"Average ROUGE-{metric} score: {values['f']}\n")



In [ ]:
# Changing Compact Ratio and adjust min_length, max_length and length penalty hyperparameters

generated_summaries, reference_summaries = [], []
# Loop over the dataset and generate summaries
print("A len \t\t ES len \t\t AS len \t\t E CP \t\t A CP")
for example in dataset["test"]:
    # Tokenize the input text and truncate it to a maximum length of 1024 tokens
    input_ids = tokenizer.encode(example["article"], max_length=1024, truncation=True, return_tensors="pt")

    # Generate the summary
    input_ids = input_ids.to(device)
    lenInput = len(example["article"])
    min_length = int((0.05*lenInput))
    max_length = int((0.10*lenInput))

    summary_ids = model.generate(input_ids, num_beams=4, length_penalty=3, min_length=min_length, max_length=max_length, no_repeat_ngram_size=3)
    summary_ids = summary_ids.to('cpu')
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    generated_summaries.append(summary)
    reference_summaries.append(example["highlights"])
    lenA = len(example["article"])
    lenES = len(example["highlights"])
    lenAS = len(summary)
    ECP = (lenA-lenES)*100/lenA
    ECP = str(round(ECP, 2))
    ACP = (lenA-lenAS)*100/lenA
    ACP = str(round(ACP, 2))
    print(lenA, "\t\t", lenES, "\t\t\t", lenAS ,"\t\t\t", ECP, "\t\t", ACP) 

scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores and their averages for the dataset
for metric, values in scores.items():
    print(f"\nROUGE-{metric} scores: \n")
    print(values)
    print(f"Average ROUGE-{metric} score: {values['f']}\n")

In [7]:
# Load the BART model
bart_config = BartConfig.from_pretrained("facebook/bart-large-cnn")
# Set the activation function of the decoder's first layer to ReLU
# bart_config.activation_function = 'relu'
# bart_config.decoder_layerdrop = 0.1
# bart_config.dropout = 0.1
# bart_config.activation_dropout = 0.0
# bart_config.decoder_attention_heads = 30
# bart_config.decoder_layers = 20
model.config = bart_config


# Set the activation function of the decoder's first layer to SiLU
#model.model.decoder.layers[0].activation_fn = ReLU()

In [8]:
# Changing Activation function to RELU (It puts all negative values as zero and remaining values as it is)
bart_config = BartConfig.from_pretrained("facebook/bart-large-cnn")
bart_config.activation_function = 'relu'
model.config = bart_config

In [11]:
# Changing Activation function to RELU (It puts all negative values as zero and remaining values as it is)
bart_config = BartConfig.from_pretrained("facebook/bart-large-cnn")
bart_config.activation_function = 'elu'
model.config = bart_config
for layer in model.model.decoder.layers:
    layer.activation_fn = ELU()

In [12]:
# Changing Activation function to mish 
class Mish(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x * torch.tanh(F.softplus(x))
        
bart_config = BartConfig.from_pretrained("facebook/bart-large-cnn")
bart_config.activation_function = 'mish'
model.config = bart_config

for layer in model.model.decoder.layers:
    layer.activation_fn = Mish()

In [13]:
# Changing Attention heads
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Change the number of attention heads in the encoder and decoder layers to 30
bart_config = model.config
bart_config.encoder_attention_heads = 30
bart_config.decoder_attention_heads = 30

# Set the updated configuration to the model
model.config = bart_config

In [14]:
# Changing attention heads to 50

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Change the number of attention heads in the encoder and decoder layers to 30
bart_config = model.config
bart_config.encoder_attention_heads = 50
bart_config.decoder_attention_heads = 50

# Set the updated configuration to the model
model.config = bart_config

In [15]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Change the number of encoder and decoder layers
bart_config = model.config
bart_config.encoder_layers = 20  # Change the number of encoder layers
bart_config.decoder_layers = 20  # Change the number of decoder layers

# Set the updated configuration to the model
model.config = bart_config

In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Create the optimizer and set its parameters
optimizer = AdamW(model.parameters(), lr=1e-6)

# Set the optimizer to the model
model.optimizer = optimizer

In [21]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Create the optimizer and set its parameters
optimizer = AdamW(model.parameters(), lr=1e-6, weight_decay=0.1)

# Set the optimizer to the model
model.optimizer = optimizer